In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import wrangle

## Acquire Data

Data acquired from 'zillow' database. Looking at the available rows and running a few basic queries in MySQL, the following decisions were made:
- bathroomcnt and bedroomcnt will be used
  - calculatedbathnbr and threequarterbathnbr had many nulls
  - roomcnt was primarily zeros and its meaning is unclear.  this could be pursued in future analyses
- taxamount won't be pulled in as it is calculated off of tax value, our target variable
- regionidzip and fips will be used
  - regionidneighborhood was half nulls and barely had more distinct entries than regionidzip


In [ ]:
#Write some SQL - expect 52441 rows
sql = """
SELECT bedroomcnt as bed,
    bathroomcnt as bath, 
    calculatedfinishedsquarefeet as sf, 
    taxvaluedollarcnt as value, 
    yearbuilt, 
    assessmentyear,
    regionidzip as zipcode, 
    fips
FROM properties_2017
    JOIN propertylandusetype USING(propertylandusetypeid)
    JOIN predictions_2017 USING(parcelid)
WHERE propertylandusedesc = 'Single Family Residential' AND transactiondate LIKE '2017%%';
"""

In [ ]:
df = df_backup = pd.read_sql(sql,wrangle.get_db_url('zillow'))

In [ ]:
df.shape #rows match

In [ ]:
df.info()

## Prepare Data
### Investigating Nulls

In [ ]:
#let's check out how the nulls overlap
#Count how many values we have per row
hist = df.count(axis=1,numeric_only=False)
plt.hist(hist[hist<8])

Looking at the above, a decent number of these are only missing one or two pieces of data. However, since we are still only looking at ~140 total rows, I am going to go ahead and drop them all. Ideally, I'd check some of this with stakeholders. My primary concern is that we may be inadvertently trimming a particular geographic area (perhaps one with poor reporting on year built and zip code)

**ACTION:** Drop all nulls

In [ ]:
df.dropna(inplace=True)
df.shape

### Look at the distributions of each feature

In [ ]:
#See on logarythmic scale to better see outliers
plt.figure(figsize=(10,20))
ct=0
for c in df.columns:
    ct +=1
    plt.subplot(8,2,ct)
    plt.hist(df[c],bins=20)
    plt.title(c+'_log')
    plt.yscale('log')
    ct +=1
    plt.subplot(8,2,ct)
    plt.hist(df[c],bins=20)
    plt.title(c)
    plt.ylim((0,100))
    
plt.tight_layout()

Definitely some bed, bath, sf, value, assessment year and year outliers. I don't want to snap any values in. 

- Since Zillow estimates are not expected to be utilized by the top 1%, I want to cut some of the expensive and large houses. We'll cut the top .1% of sf homes off the dataset. NOTE: I recognized that sf is being used as a proxy for value, but it's one method of avoiding trimming by our target so that we can better specify how the model was trained in a useful manner
- Since the data is skewed right, I want to trim less off the left. After considering a few different cutoffs, I found that california code restricts minimum dwelling size to 120 sq ft.
  - "Every dwelling unit shall have at least one room that shall have not less than 120 square feet (13.9 m2) of net floor area"
  
**ACTION:** Drop rows with the top .1% of sf or an sf of less than 120

In [ ]:
#drop top 1% of sf
df = df[df.sf<df.sf.quantile(.999)]

#drop anything less than 120 sf
df = df[df.sf>=120]
df.shape

In [ ]:
df.describe().T

In [ ]:
#Plot again
plt.figure(figsize=(10,15))
ct=0
for c in df.columns:
    ct +=1
    plt.subplot(8,2,ct)
    plt.hist(df[c],bins=20)
    plt.title(c+'_log')
    plt.yscale('log')
    ct +=1
    plt.subplot(8,2,ct)
    plt.hist(df[c],bins=20)
    plt.title(c)
    plt.ylim((0,50))
    
plt.tight_layout()

Even after using sf trimming, we still have quite a few outliers in each category. Because of that, I'll do more trimming on the high side. Since Zillow's target customers aren't the super rich, I feel comfortable trimming 9+ bedrooms, 9+ bathrooms and 5+ million value. While we wanted to avoid any triming by value, it is import to get rid of these extreme outliers.

In [ ]:
df[df.bed > 8].shape

In [ ]:
df[df.bath>8].shape

In [ ]:
df[df.value>5_000_000].shape

In [ ]:
#see percent that fall in this category
df[(df.value>5_000_000) | (df.bath >8) | (df.bed >8)].shape[0] / df.shape[0]

Even with the extra trimming, that only accounts for <1% of the data.

**Action:** Drop all rows with 9+ beds, 9+ baths, or a value of 5+ million.



In [ ]:
df = df[(df.value < 5_000_000) & (df.bath < 9) & (df.bed <9)]
df.shape

Now let's look at assessment year & zip code

In [ ]:
df.assessmentyear.value_counts()

Since there was a transaction it looks to have a current assessment.

**ACTION:** Drop the assessment year column

In [ ]:
df.zipcode.value_counts()

In [ ]:
#see if we don't have enough zipcode information for some
df.zipcode.value_counts().hist(bins=100)
plt.xlim(0,100)

There are a few zip codes with very few datapoints. 

**ACTION:** Drop all rows with a zip code that has less than 50 other properties in that zip code.



In [ ]:
zip_cnt = df.zipcode.value_counts()
zip_cnt

In [ ]:
zip_cnt[zip_cnt < 30].sum()/df_backup.shape[0] #1.3% of total dataset

In [ ]:
drp_zips = zip_cnt[zip_cnt < 30].index
drp_zips

**WARNING:** I looked up some of these zip codes, and they do not correspond to the fips (counties) that are in this dataset.  Looking at the metadata, it states that these *should* be zip codes.  I will keep this info for now, as there is no reason to assume it is less or more valid than the fips.  It is possible it is another geographic identifier unique to zillow.

Ideally, I would import this information into ArcGIS and plot the lat/lon of these properties and colorize them by the 'regionidzip'.  This would help to confirm they are geographic properties.

In [ ]:
df.shape

In [ ]:
#drop the rows with those zip codes
df = df[df.zipcode.isin(drp_zips)==False]
df.shape

### Check dataypes are appropriate

In [ ]:
df.bed.value_counts() #can be integer

In [ ]:
(df.yearbuilt % 1).value_counts() # can be integer

In [ ]:
(df.sf %1).value_counts() # can be integer

In [ ]:
(df.value %1).value_counts() # can be integer

In [ ]:
df[df.zipcode % 1 != 0] #good, all zips are whole numbers.  convert to string

While sf and value can be floats, realistically they rarely are and the partial sf and dollars wouldn't matter.

**ACTION:** Round down sf and value, then convert bed, yearbuilt, sf, value and zipcode to integers.

In [ ]:
df.bed = df.bed.astype(int)
df.yearbuilt = df.yearbuilt.astype(int)
#astype automatically rounds floats
df.sf = df.sf.astype(int)
df.value = df.value.astype(int)
#int first to get rid of the ".0" then to string
df.zipcode = df.zipcode.astype(int).astype(str)
df.info()

### Now create a new feature 

I want a new column that is squarefeet per # of bedrooms.

Since the number of rooms is likely related to the squarefootage, I want to try and create a combined column.  Because bathrooms are generally much smaller than bedrooms, I don't want to include that in the numerator.  

In [ ]:
#create sf per bed column, consider 0 beds to have 1 bedroom (EX: studio)
for i in df.index:
    #for each row do math of sf/bed.  If bed is zero, use 1
    df.loc[i,'sf_per_bed'] = df.loc[i,'sf']/ max(1,df.loc[i,'bed'])
df.info()

In [ ]:
df.describe().T

In [ ]:
df.sf_per_bed.hist()
plt.ylim(0,100)

In [ ]:
df[df.sf_per_bed < 200]
#looks like we have an outlier - likely typo - index #48784

In [ ]:
df[df.sf_per_bed >3500]

**ACTION:** Drop the significant outliers in this column.  sf_per_bed < 100 and sf_per_bed >= 3500

In [ ]:
df = df[(df.sf_per_bed>99) & (df.sf_per_bed<3500)] #only gets rid of 3

### Now encode the categorical variables, drop and reorder columns

In [ ]:
#I want to map then encode so that I have common sense 
# names for EDA and easy to read columns for the model

#map to county names
df['county'] = df.fips.map({6037: 'LosAngeles_CA',6059:'Orange_CA',6111:'Ventura_CA'})
#encode into dummy df
d_df = pd.get_dummies(df['county'],drop_first=True)
#concat dummy df to the rest
df = pd.concat([df,d_df],axis=1)

In [ ]:
df.info()

In [ ]:
#Get rid of original fips column 
df.drop(columns=['fips','assessmentyear'],inplace=True)
#will also just not select assesment year in module

In [ ]:
#Reorder now, prior to encoding zipcode
df.columns

In [ ]:
#reorder columns with target and categorical in the front, encoded at the back
df = df.reindex(columns=['value', 'zipcode', 'county', 'bed', 'bath', 'sf', 'sf_per_bed', 'yearbuilt', 'Orange_CA', 'Ventura_CA'])

In [ ]:
df.info()

##### zip code encoding

In [ ]:
#In my function, I'll probably want to make including zip a parameter.
#However, still want to encode for now
#encode into dummy df
dz_df = pd.get_dummies(df['zipcode'],drop_first=True)
#concat dummy df to the rest
df = pd.concat([df,dz_df],axis=1)

In [ ]:
df.info() #342 columns now

## Drop all into wrangle function in wrangle.py

### Test the function

In [2]:
test_df = wrangle.getZillowData()

In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52441 entries, 0 to 52440
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   bed        52441 non-null  float64
 1   bath       52441 non-null  float64
 2   sf         52359 non-null  float64
 3   value      52440 non-null  float64
 4   yearbuilt  52325 non-null  float64
 5   zipcode    52415 non-null  float64
 6   fips       52441 non-null  float64
dtypes: float64(7)
memory usage: 3.2 MB


In [4]:
tr, te, val = wrangle.prep_zillow(test_df,include_zip=False)

In [5]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36448 entries, 17988 to 40159
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       36448 non-null  int64  
 1   county      36448 non-null  object 
 2   bed         36448 non-null  int64  
 3   bath        36448 non-null  float64
 4   sf          36448 non-null  int64  
 5   sf_per_bed  36448 non-null  float64
 6   yearbuilt   36448 non-null  int64  
 7   Orange_CA   36448 non-null  uint8  
 8   Ventura_CA  36448 non-null  uint8  
dtypes: float64(2), int64(4), object(1), uint8(2)
memory usage: 2.3+ MB


In [6]:
tr, te, val = wrangle.prep_zillow(test_df)

In [7]:
tr.columns[0:12]

Index(['value', 'zipcode', 'county', 'bed', 'bath', 'sf', 'sf_per_bed',
       'yearbuilt', 'Orange_CA', 'Ventura_CA', '95983', '95984'],
      dtype='object')

In [8]:
tr[['value', 'zipcode', 'county', 'bed', 'bath', 'sf', 'sf_per_bed', 'yearbuilt',
       'Orange_CA', 'Ventura_CA', '95983', '95984']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35984 entries, 33376 to 19456
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       35984 non-null  int64  
 1   zipcode     35984 non-null  object 
 2   county      35984 non-null  object 
 3   bed         35984 non-null  int64  
 4   bath        35984 non-null  float64
 5   sf          35984 non-null  int64  
 6   sf_per_bed  35984 non-null  float64
 7   yearbuilt   35984 non-null  int64  
 8   Orange_CA   35984 non-null  uint8  
 9   Ventura_CA  35984 non-null  uint8  
 10  95983       35984 non-null  uint8  
 11  95984       35984 non-null  uint8  
dtypes: float64(2), int64(4), object(2), uint8(4)
memory usage: 2.6+ MB


### DATA PREP SUMMARY:
- 2% of the data was dropped.  Rows were dropped if:
  - There were any nulls
  - It fell in the top 1% of square footage
  - It had < 120 square feet 
  - There were less than 30 rows with the same zip code
  - There were 9+ beds or 9+ baths
  - The value was >= 5 million
- Bed, yearbuilt, square footage and value were converted to integers
- zipcode was converted to a string
- fips was mapped to a readable column
- zipcode and fips were encoded
- Created a new column sf/bed
  - bedrooms of 0 were treated as bed of 1
  - major outliers were trimmed (only 3 rows)
- columns were reorganized so that target and categorical were at the beginning, with encoded columns at the end